In [143]:
import numpy as np
import pandas as pd
import plotly as py
import plotly.express as px

# Loading csv files into a pandas DataFrame
db1 = pd.read_csv('TLC_New_Driver_App.csv')# TLC new apps: Mostly contains data from 2020-2022
db2 = pd.read_csv('Historical_Driver_Application_Status.csv')# TLC apps: Contains more data from 2015-2022
db3 = pd.read_csv('FHV_Base_Aggregate_Report.csv')# Base trip reporting
TLC = pd.concat([db1,db2])# Combning both TLC new apps datasets
TLC.head()
db3.head()

ValueError: time data 'December' does not match format '%M' (match)

In [147]:
FHV = db3.groupby(['Year','Month','Month Name'])['Unique Dispatched Vehicles'].sum().reset_index()
print(FHV.head(20))


    Year  Month Month Name  Unique Dispatched Vehicles
0   2015      1    January                       20345
1   2015      2   February                       21171
2   2015      3      March                       22137
3   2015      4      April                       30986
4   2015      5        May                       28296
5   2015      6       June                       33871
6   2015      7       July                       36474
7   2015      8     August                       42880
8   2015      9  September                       50802
9   2015     10    October                       62224
10  2015     11   November                       64121
11  2015     12   December                       65390
12  2016      1    January                       68794
13  2016      2   February                       69265
14  2016      3      March                       66612
15  2016      4      April                       74565
16  2016      5        May                       80903
17  2016  

In [149]:
fig = px.bar(FHV, x='Year', y='Unique Dispatched Vehicles',\
             labels={'Year':'Year'},\
             color='Month Name', barmode='group',text = 'Unique Dispatched Vehicles',\
             height=400)
fig.show()

fig2 = px.box(FHV, x='Year', y='Unique Dispatched Vehicles',\
             #labels={'x':'App Date (M)','Number of Applicants':'No. of Apps'},\
             #color=df5['Driver Exam'],\
             height=400)
fig2.show()

In [106]:
TLC = TLC.rename(columns={'App Date': 'App Date (M/D/Y)'}) #Changing column name to include (m/d/Y)
print(TLC.info())#Some info about the dataframe
TLC.head()#first 5 rows

<class 'pandas.core.frame.DataFrame'>
Int64Index: 153081 entries, 0 to 145972
Data columns (total 12 columns):
 #   Column                   Non-Null Count   Dtype 
---  ------                   --------------   ----- 
 0   App No                   153081 non-null  int64 
 1   Type                     153081 non-null  object
 2   App Date (M/D/Y)         153080 non-null  object
 3   Status                   153081 non-null  object
 4   FRU Interview Scheduled  153081 non-null  object
 5   Drug Test                153081 non-null  object
 6   WAV Course               153081 non-null  object
 7   Defensive Driving        153081 non-null  object
 8   Driver Exam              153081 non-null  object
 9   Medical Clearance Form   153081 non-null  object
 10  Other Requirements       153081 non-null  object
 11  Last Updated             153081 non-null  object
dtypes: int64(1), object(11)
memory usage: 15.2+ MB
None


,App No,Type,App Date (M/D/Y),Status,FRU Interview Scheduled,Drug Test,WAV Course,Defensive Driving,Driver Exam,Medical Clearance Form,Other Requirements,Last Updated
0,5990450,HDR,02/17/2022,Incomplete,Not Applicable,Complete,Complete,Complete,Needed,Complete,Not Applicable,04/01/2022 05:00:03 PM
1,5961459,HDR,02/26/2020,Incomplete,Not Applicable,Needed,Needed,Complete,Needed,Needed,Copy of DMV license needed,04/01/2022 05:00:03 PM
2,5989657,HDR,02/02/2022,Incomplete,Not Applicable,Complete,Complete,Complete,Needed,Complete,Not Applicable,04/01/2022 05:00:03 PM
3,5992972,HDR,03/29/2022,Incomplete,Not Applicable,Needed,Needed,Needed,Needed,Needed,Fingerprints needed; Copy of DMV license needed,04/01/2022 05:00:03 PM
4,5990896,HDR,02/24/2022,Incomplete,Not Applicable,Needed,Needed,Needed,Needed,Needed,Fingerprints needed; Copy of DMV license needed,04/01/2022 05:00:03 PM


In [107]:
# Converts App Date column into a datetime instance column. Note that once the conversion is done the format is 
# changed to (Y/M/D)
TLC['App Date (M/D/Y)'] = pd.to_datetime(TLC['App Date (M/D/Y)'],format = '%m/%d/%Y')
# Renaming to reflect the fact that 'to_dataframe' change the order of the display date to (Y/M/D)
TLC = TLC.rename({'App Date (M/D/Y)':'App Date (Y/M/D)'}, axis = 1)
TLC.head()

,App No,Type,App Date (Y/M/D),Status,FRU Interview Scheduled,Drug Test,WAV Course,Defensive Driving,Driver Exam,Medical Clearance Form,Other Requirements,Last Updated
0,5990450,HDR,2022-02-17,Incomplete,Not Applicable,Complete,Complete,Complete,Needed,Complete,Not Applicable,04/01/2022 05:00:03 PM
1,5961459,HDR,2020-02-26,Incomplete,Not Applicable,Needed,Needed,Complete,Needed,Needed,Copy of DMV license needed,04/01/2022 05:00:03 PM
2,5989657,HDR,2022-02-02,Incomplete,Not Applicable,Complete,Complete,Complete,Needed,Complete,Not Applicable,04/01/2022 05:00:03 PM
3,5992972,HDR,2022-03-29,Incomplete,Not Applicable,Needed,Needed,Needed,Needed,Needed,Fingerprints needed; Copy of DMV license needed,04/01/2022 05:00:03 PM
4,5990896,HDR,2022-02-24,Incomplete,Not Applicable,Needed,Needed,Needed,Needed,Needed,Fingerprints needed; Copy of DMV license needed,04/01/2022 05:00:03 PM


In [108]:
# Sorts DF by app Date: Note that if the column isn't previously converted to 
# a date-time format using "to_datetime" then it will take the values as char
# and will not sort by date.
TLC = TLC.sort_values(by = 'App Date (Y/M/D)',na_position='first').dropna()
print(TLC.info())
TLC.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 153080 entries, 6603 to 1691
Data columns (total 12 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   App No                   153080 non-null  int64         
 1   Type                     153080 non-null  object        
 2   App Date (Y/M/D)         153080 non-null  datetime64[ns]
 3   Status                   153080 non-null  object        
 4   FRU Interview Scheduled  153080 non-null  object        
 5   Drug Test                153080 non-null  object        
 6   WAV Course               153080 non-null  object        
 7   Defensive Driving        153080 non-null  object        
 8   Driver Exam              153080 non-null  object        
 9   Medical Clearance Form   153080 non-null  object        
 10  Other Requirements       153080 non-null  object        
 11  Last Updated             153080 non-null  object        
dtypes: datetime64[n

,App No,Type,App Date (Y/M/D),Status,FRU Interview Scheduled,Drug Test,WAV Course,Defensive Driving,Driver Exam,Medical Clearance Form,Other Requirements,Last Updated
6603,5921774,PDR,1997-04-28,Incomplete,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Fingerprints & other items needed. Visit www.n...,04/01/2022 05:00:03 PM
115868,5921775,PDR,1997-04-28,Denied,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Fingerprints & other items needed. Visit www.n...,04/08/2020 11:00:03 PM
2880,5921773,PDR,1997-04-28,Incomplete,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Fingerprints & other items needed. Visit www.n...,04/01/2022 05:00:03 PM
411,5921772,PDR,1997-04-28,Incomplete,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Fingerprints & other items needed. Visit www.n...,04/01/2022 05:00:03 PM
116421,5921776,PDR,1997-04-28,Denied,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Not Applicable,Fingerprints needed,04/20/2020 11:00:00 PM


In [109]:
# We drop all rows prior to 2015 since the data base doesn't many values prior to 2015
TLC1 = TLC[TLC['App Date (Y/M/D)'].dt.year > 2015]
print(TLC1.info())
TLC1.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 153051 entries, 10489 to 1691
Data columns (total 12 columns):
 #   Column                   Non-Null Count   Dtype         
---  ------                   --------------   -----         
 0   App No                   153051 non-null  int64         
 1   Type                     153051 non-null  object        
 2   App Date (Y/M/D)         153051 non-null  datetime64[ns]
 3   Status                   153051 non-null  object        
 4   FRU Interview Scheduled  153051 non-null  object        
 5   Drug Test                153051 non-null  object        
 6   WAV Course               153051 non-null  object        
 7   Defensive Driving        153051 non-null  object        
 8   Driver Exam              153051 non-null  object        
 9   Medical Clearance Form   153051 non-null  object        
 10  Other Requirements       153051 non-null  object        
 11  Last Updated             153051 non-null  object        
dtypes: datetime64[

,App No,Type,App Date (Y/M/D),Status,FRU Interview Scheduled,Drug Test,WAV Course,Defensive Driving,Driver Exam,Medical Clearance Form,Other Requirements,Last Updated
10489,5671714,HDR,2016-01-05,Approved - License Issued,Not Applicable,Complete,Complete,Complete,Complete,Complete,Not Applicable,09/27/2016 06:11:27 PM
2138,5671835,HDR,2016-01-05,Approved - License Issued,Not Applicable,Complete,Complete,Complete,Complete,Complete,Not Applicable,08/27/2016 06:11:23 PM
7090,5674592,HDR,2016-01-15,Denied,Not Applicable,Complete,Complete,Complete,Needed,Complete,Open items needed. E-mail newdriverapp@tlc.nyc...,07/09/2016 06:11:32 PM
5324,5674622,CDR,2016-01-15,Approved - License Issued,Not Applicable,Complete,Complete,Complete,Complete,Complete,Not Applicable,07/05/2016 06:11:21 PM
3674,5674712,CDR,2016-01-15,Denied,Not Applicable,Complete,Complete,Complete,Needed,Complete,Not Applicable,07/02/2016 06:11:28 PM


In [110]:
# Creating TLC2 which has items grouped by Driver Exam and Year
group = TLC1.groupby(['Driver Exam',pd.Grouper(key = 'App Date (Y/M/D)', freq = 'Y')])
# Note: we also rename the App No column since after we aggregate we are 
# left with the total No. of Apps. 
TLC2 = group['App No'].count().reset_index().rename({'App No': 'No. of Apps'}, axis = 1)
print(TLC2.info())
TLC2.head()
#------------------------------------------------------------------------------------------------------------
# for key, values in group2:
#     print(key)
#     print('-'*70)
#     print(values)
#     print('\n')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21 entries, 0 to 20
Data columns (total 3 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   Driver Exam       21 non-null     object        
 1   App Date (Y/M/D)  21 non-null     datetime64[ns]
 2   No. of Apps       21 non-null     int64         
dtypes: datetime64[ns](1), int64(1), object(1)
memory usage: 632.0+ bytes
None


,Driver Exam,App Date (Y/M/D),No. of Apps
0,Complete,2016-12-31,26309
1,Complete,2017-12-31,34732
2,Complete,2018-12-31,29837
3,Complete,2019-12-31,10214
4,Complete,2020-12-31,3145


In [111]:
# Creating TLC3 which adds a column of percent total "Completed|Needed|Not Applicable" per year
my_group = TLC2.groupby([pd.Grouper(key = 'App Date (Y/M/D)', freq = 'Y')])
temp = my_group[['No. of Apps']].sum()
TLC3 = pd.merge(TLC2, temp, on='App Date (Y/M/D)',suffixes = ('',' (Total)'))
TLC3['Percent'] = TLC3['No. of Apps'] / TLC3['No. of Apps (Total)']
TLC3.head(20)

,Driver Exam,App Date (Y/M/D),No. of Apps,No. of Apps (Total),Percent
0,Complete,2016-12-31,26309,37114,0.708870
1,Needed,2016-12-31,10578,37114,0.285014
2,Not Applicable,2016-12-31,227,37114,0.006116
3,Complete,2017-12-31,34732,43279,0.802514
4,Needed,2017-12-31,8322,43279,0.192287
5,Not Applicable,2017-12-31,225,43279,0.005199
6,Complete,2018-12-31,29837,35532,0.839722
7,Needed,2018-12-31,5573,35532,0.156845
8,Not Applicable,2018-12-31,122,35532,0.003434
9,Complete,2019-12-31,10214,13162,0.776022


In [112]:
# Making a line plot of df3
my_fig = px.line(TLC3, x=TLC3['App Date (Y/M/D)'].dt.year, y='Percent',\
             labels={'x':'Year'},\
             color=df3['Driver Exam'],\
             height=400)
my_fig.show()

In [113]:
# Creating df4 which has items grouped by Driver Exam and Month
group2 = TLC1.groupby(['Driver Exam',TLC1['App Date (Y/M/D)'].dt.month])
TLC4 = group2['App No'].count().reset_index().rename({'App No': 'No. of Apps'}, axis = 1)
TLC4.head(20)

,Driver Exam,App Date (Y/M/D),No. of Apps
0,Complete,1,9628
1,Complete,2,8320
2,Complete,3,9590
3,Complete,4,8901
4,Complete,5,9832
5,Complete,6,10417
6,Complete,7,10810
7,Complete,8,11720
8,Complete,9,10156
9,Complete,10,10996


In [115]:
# Creating df5 which adds a column of percent total "Completed|Needed|Not Applicable" per month
my_group2 = TLC4.groupby(['App Date (Y/M/D)'])
temp2 = my_group2[['No. of Apps']].sum()
temp2.head()
TLC5 = pd.merge(TLC4, temp2, on='App Date (Y/M/D)',suffixes = ('',' (Total)'))
TLC5['Percent'] = TLC5['No. of Apps'] / TLC5['No. of Apps (Total)']
TLC5.head(40)

,Driver Exam,App Date (Y/M/D),No. of Apps,No. of Apps (Total),Percent
0,Complete,1,9628,12089,0.796427
1,Needed,1,2403,12089,0.198776
2,Not Applicable,1,58,12089,0.004798
3,Complete,2,8320,10871,0.765339
4,Needed,2,2484,10871,0.228498
5,Not Applicable,2,67,10871,0.006163
6,Complete,3,9590,14044,0.682854
7,Needed,3,4392,14044,0.312731
8,Not Applicable,3,62,14044,0.004415
9,Complete,4,8901,11654,0.763772


In [152]:
# Making a line plot of TLC5
my_fig = px.line(TLC5, x=TLC5['App Date (Y/M/D)'], y='Percent',\
             labels={'App Date (Y/M/D)':'Month'},\
             color=df5['Driver Exam'],\
             height=400)
my_fig.show()

In [117]:
# Making bar plot of TLC3 showing No. of Apps (y-axis) vs App Date Year (x-axis) and grouped by Driver Exam 
fig = px.bar(TLC3, x=TLC3['App Date (Y/M/D)'].dt.year, y='No. of Apps',\
             labels={'x':'App Date (Y)'},\
             color='Driver Exam', barmode='group',text = TLC3['No. of Apps'],\
             height=400)
fig.show()

In [63]:
#  Making box plot of TLC5 showing No. of Apps (y-Axis) vs Month (x-axis) and grouped by Driver Exam 
fig2 = px.box(df5, x=df5['App Date (Y/M/D)'].dt.month, y='Number of Applicants',\
             labels={'x':'App Date (M)','Number of Applicants':'No. of Apps'},\
             color=df5['Driver Exam'],\
             height=400)
fig2.show()